In [1]:
#'''>
#Todo: test with transaction and inventory data from the same date..
#
#'''


In [2]:
# Import
import pandas as pd
import numpy as np
import os
from datetime import date

In [3]:
# Read starting inventory file


### Open transaction data
The transaction file is obtained in Horne web page/ Applications/ Rapports Horne/ MAIT/ SIGI/ Transactions de materiel.  
Take the transactions from yesterday to yesterday.  .  
To self: possibly use all transactions starting from a certain date. Accounts for transaction changes.

In [4]:
# Read transaction file.
dT = pd.read_csv(r'C:\Users\markz\OneDrive\Documents\Fonte_data\transaction_data_2022-03-28.csv')
#r'C:\Users\markz\OneDrive\Documents\Fonte_data\transaction_data_2022-03-28.csv'
# Drop useless columns
dT = dT.loc[:,['TAS_ID', 'REVERSE_TAS_ID1', 'Textbox58', 'SHIPPER_CODE','ACTION_TIME']]
# Rename columns
dT.columns = ['sourcePort', 'destPort', 'DMT', 'shipperCode','time']
# Sort by time
dT = dT.sort_values(by=['time'])
# Bring transactions to 564, 565 to the top. These are sourcePorts for 916, 917 and must not be empty.
temp = dT[dT['destPort'].isin([564, 565])]
dT.drop(dT[dT['destPort'].isin([564, 565])].index, inplace=True)
dT = pd.concat([temp,dT], axis=0)

dT.head(15)

,sourcePort,destPort,DMT,shipperCode,time
77,304,564,"380,48",,61509
82,373,564,"31,56",,61521
104,600,744,"16,04",600,633
105,600,804,"2,55",600,633
23,20001,385,"30,57",INC4,12758
30,20001,482,"29,64",DDG,20438
137,841,810,"13,52",841,21136
132,841,810,"4,78",841,22441
139,841,831,"6,61",841,23252
32,20001,566,"12,48",FDN,23543


In [5]:
# Convert comma to decimal
dT.replace(',','.', regex=True, inplace=True)
# Convert to float
dT.loc[:,'DMT'] = pd.to_numeric(dT.loc[:,'DMT'])
dT.head()

,sourcePort,destPort,DMT,shipperCode,time
77,304,564,380.48,,61509
82,373,564,31.56,,61521
104,600,744,16.04,600,633
105,600,804,2.55,600,633
23,20001,385,30.57,INC4,12758


In [6]:
# Keep rows if destination is to "Eglise"
# Todo eglisePorts read from csv
eglisePorts = [564,565,900,903,906,910,913,915,916,917,918,919,920,921,922,923,924,925]

dT = dT[dT['destPort'].isin(eglisePorts)]
dT.head()

,sourcePort,destPort,DMT,shipperCode,time
77,304,564,380.48,,61509
82,373,564,31.56,,61521
67,20002,906,85.52,EAGL,55246
68,20002,906,87.20,EAGL,55256
62,20002,900,95.82,MTA,55351


We want a unique identifier for each material, therefore we wish to convert the port numbers to an alphabetical shipper code.
shipperCode column currently contains both string shipper codes ('EAGL', 'STAL', etc.) and port numbers (812, 376, etc.). 

In [7]:
# shipperCode column contains port numbers and shipper codes
# Replace empty strings with NaN
#dT['shipperCode'] = dT['shipperCode'].replace(r'^\s*$', np.nan, regex=True)
#dT.head()

Now we load a dictionnary to assign a shipper code to each line of dT that has a port number value in the column 'shipperCode'.  
However, some port numbers are actually a mix of materials (ex: 304 is a mix of green concentrates, 385 is a mix of concentrates with nickel).  
If a port is composed of a single type of material, we treat it as a single material even if it is a mix of different materials (ex: all 304 is MTA despite having MTA, XKM, BCM, COPA, FLI).  
If not, we consider its composition (ex: 564 can be a mix of concentrates, pyrite and electronics).


In [8]:
# Load the dictionnary to replace numerical port numbers with shipper codes. shipper_code = f(source_port)
# This dictionnary must be up to date, as port numbers and materials may change.

shipID = pd.read_csv(r'C:\Users\markz\OneDrive\Documents\Fonte_data\sourcePort_to_shipperId.csv')
#r''
shipID.head()


,ShipperID,SourcePort
0,BCM,NaN
1,Pyrite_don_rouyn_gallen,498.0
2,Pyrite_don_rouyn_gallen,114.0
3,Pyrite_don_rouyn_gallen,497.0
4,Pyrite_don_rouyn_gallen,499.0


In [9]:
# Remove row if SourcePort value is missing
shipID = shipID.loc[shipID['SourcePort'].notnull(),:]
shipID.shape
shipID

,ShipperID,SourcePort
1,Pyrite_don_rouyn_gallen,498.0
2,Pyrite_don_rouyn_gallen,114.0
3,Pyrite_don_rouyn_gallen,497.0
4,Pyrite_don_rouyn_gallen,499.0
5,NMP,520.0
...,...,...
88,Fine Carbone,778.0
89,IMMSA,593.0
90,NAFT,492.0
91,CCR Residu,572.0


In [10]:
# Create dictionary
shipID_dict = shipID.set_index('SourcePort')['ShipperID'].to_dict()
# This also works
# shipID_dict = dict(zip(shipID['SourcePort'], shipID['ShipperID']))

In [11]:
# Map shipperID to transactions
dT['shipperID'] = dT['sourcePort'].map(shipID_dict)
# Drop null values of shipperID
#dT = dT[dT['shipperID'].notnull()]
dT.head()
before = dT.copy()

Copy and overwrite alphabetical values from shipperCode to shipperID. shipperCode data is more reliable because it comes directly from the transactions data, whereas shipperID uses a manually updated dictionnary.


In [12]:
dT.loc[(~dT['shipperCode'].str.isnumeric()) & (~dT['shipperCode'].str.contains(r'^\s*$')), 'shipperID']\
= dT.loc[(~dT['shipperCode'].str.isnumeric()) & (~dT['shipperCode'].str.contains(r'^\s*$')), 'shipperCode']

In [13]:
# Show rows where an update was made
a = before[dT['shipperID'] != before['shipperID']]['shipperID']
b = dT[dT['shipperID'] != before['shipperID']]['shipperID']
comparison = pd.concat([a,b],axis=1)
comparison.set_axis(['before_shipperID','after_shipperID'], axis=1, inplace=True)
comparison.head()

,before_shipperID,after_shipperID
67,NaN,EAGL
68,NaN,EAGL
62,NaN,MTA
63,NaN,MTA
64,NaN,FLI


In [14]:
# if shipperID is null, and if shipperCode is notnull, copy shipperCode to shipperID
# Overwrites the shipperCode given by the transaction data.
# This also copies over numerical ports to shipperID, which is not wanted.
dT['shipperID'] = np.where(dT['shipperCode'].notnull() & dT['shipperID'].isnull(),dT['shipperCode'], dT['shipperID'])
dT.head()

,sourcePort,destPort,DMT,shipperCode,time,shipperID
77,304,564,380.48,,61509,MTA
82,373,564,31.56,,61521,Electronique Rx
67,20002,906,85.52,EAGL,55246,EAGL
68,20002,906,87.20,EAGL,55256,EAGL
62,20002,900,95.82,MTA,55351,MTA


In [15]:
# Check if there are numeric values in shipperID. There should be none. If there are, review the sourcePort_to_shipperID file.
dT[dT['shipperID'].str.isnumeric()]

,sourcePort,destPort,DMT,shipperCode,time,shipperID


In [16]:
# Check if there are empty. There 
dT[dT['shipperID']==' ']

,sourcePort,destPort,DMT,shipperCode,time,shipperID


In [17]:
dT

,sourcePort,destPort,DMT,shipperCode,time,shipperID
77,304,564,380.48,,61509,MTA
82,373,564,31.56,,61521,Electronique Rx
67,20002,906,85.52,EAGL,55246,EAGL
68,20002,906,87.20,EAGL,55256,EAGL
62,20002,900,95.82,MTA,55351,MTA
63,20002,900,95.73,MTA,55403,MTA
64,20002,900,86.27,FLI,55417,FLI
97,564,916,351.56,,61526,Melange fil
96,564,916,16.06,,61532,Melange fil
79,337,913,36.08,,61538,Internes


In [18]:
dT.groupby('destPort').sum()['DMT']

destPort
564    412.04
900    466.12
903     51.47
906    172.72
913     99.35
915     34.30
916    367.62
918    285.94
919     82.22
920     83.07
921    314.38
923     23.44
924     71.36
925    268.38
Name: DMT, dtype: float64

Read inventory.   
This is the starting inventory at Eglise, on the Monday of the week. Data must be entered manually. Names on the first column must exactly match the shipperID nomenclature, otherwise transactions will be omitted.

In [19]:
# Read prior inventory
inv = pd.read_csv(r'C:\Users\markz\OneDrive\Documents\Fonte_data\inv2.csv', index_col=0)
#r'C:\Users\markz\OneDrive\Documents\Fonte_data\inv2.csv'
#r'C:\Users\win7\Documents\Fonte_data\inv2.csv'
inv = inv.fillna(0)

# Debug
#inv.loc[:,:]=0
inv

,564,560,900,903,906,910,913,915,916,917,918,919,920,921,922,923,924,925
XKM,0.0,0.0,693.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
FLI,0.0,0.0,362.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MTA,0.0,0.0,711.0,0.0,0.0,0.0,0.0,0.0,126.0,378.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
COPA,0.0,0.0,380.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NAFT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,0.0,0.0,0.0,30.0,58.0,0.0,0.0,0.0,0.0,0.0
Pyrite_don_rouyn_gallen,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,150.0,0.0,0.0,0.0
BCM,0.0,0.0,154.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DUM,0.0,0.0,0.0,60.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NVA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
LSN_et_INC4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Add transactions to Eglise inventory, from oldest to newest. Error if a transactions from 564 and 564 is empty.

In [20]:
# Add transactions to inventory
# This is the naive method, find a more elegant way later.
for k in range(dT.shape[0]):
    for j in inv.columns:
        #If the sourcePort is 564, and the destPort matches with inv column, consider the composition of 564
        if dT.iloc[k,0] in [564,565] and str(dT.iloc[k,1])==j:
            # Remaining mass fraction in 564
            m = 1 - dT.loc[dT.index[k], 'DMT'] / inv.loc[:,'564'].sum()
            # Add mass to destPort
            inv.loc[:,j] += (1-m)*inv.loc[:,'564']
            print('Transferred {:.2f} tons from 564 to {}.'.format((1-m)*inv.loc[:,'564'].sum(),j))
            # Remove mass from 564
            inv.loc[:,'564'] = m * inv.loc[:,'564']
            
        else:
            for i in inv.index:
                # If shipperID and port number matches
                if dT.iloc[k,-1] == i and str(dT.iloc[k,1])==j:
                    inv.loc[i,j] += float(dT.iloc[k,2])
                    print('Added '+str(dT.iloc[k,2])+' tons of '+str(i)+' to port '+str(j)+'.')

            
                
inv.head()


Added 380.48 tons of MTA to port 564.
Added 31.56 tons of Electronique Rx to port 564.
Added 85.52 tons of EAGL to port 906.
Added 87.2 tons of EAGL to port 906.
Added 95.82 tons of MTA to port 900.
Added 95.73 tons of MTA to port 900.
Added 86.27 tons of FLI to port 900.
Transferred 351.56 tons from 564 to 916.
Transferred 16.06 tons from 564 to 916.
Added 36.08 tons of Internes to port 913.
Added 33.64 tons of IMMSA to port 913.
Added 8.52 tons of CEZ to port 915.
Added 9.76 tons of NAFT to port 915.
Added 16.02 tons of NMP to port 915.
Added 71.12 tons of NVA to port 918.
Added 23.88 tons of HGC to port 918.
Added 24.92 tons of FAS to port 918.
Added 108.64 tons of STAL to port 918.
Added 17.38 tons of STAL to port 918.
Added 40.0 tons of IMMSA to port 918.
Added 5.14 tons of UTE to port 919.
Added 61.26 tons of FAS to port 919.
Added 11.14 tons of CUZ to port 919.
Added 4.68 tons of CCR Residu to port 919.
Added 44.62 tons of NAFT to port 920.
Added 68.6 tons of NMP to port 921.
Ad

,564,560,900,903,906,910,913,915,916,917,918,919,920,921,922,923,924,925
XKM,0.000000,0.0,693.00,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
FLI,0.000000,0.0,448.27,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
MTA,41.017672,0.0,1090.85,0.0,0.0,0.0,0.0,0.00,465.462328,378.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
COPA,0.000000,0.0,380.00,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
NAFT,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,22.76,0.000000,0.0,0.0,30.0,102.62,0.0,0.0,0.0,0.0,0.0


In [21]:
# Check if any of the values are negative.
(inv.values<0).any()

False

In [22]:
inv.sum(axis=0)

564      44.42
560       0.00
900    2766.12
903     111.47
906     572.72
910       0.00
913     436.35
915     284.30
916     508.62
917     424.00
918     389.94
919     502.22
920     317.07
921     334.38
922     150.00
923     147.44
924     178.36
925     318.38
dtype: float64

In [23]:
# Read smelted transactions. This data is from the daily reactor report (rapport du reacteur)
# "Name" column for indication only. Matching uses "SourcePort" values.
smelt = pd.read_csv(r'C:\Users\markz\OneDrive\Documents\Fonte_data\smelt_2022-03-28.csv')
smelt.dropna(subset=['DMT'],inplace=True)
smelt['DMT'].sum()
smelt

,Name,SourcePort,DMT,Comment
1,Melange fil,916,704.4,NaN
2,Vert conc,900,120.5,NaN
4,Toromocho,919,253.2,NaN
5,Inco concentre,906,546.2,NaN
6,Stall,918,418.4,NaN
8,Electronique moyen carbone,924,68.4,NaN
10,CEZ,915,43.0,NaN
11,NMP,920,49.9,NaN
13,Cuzcatlan,921,418.4,NaN
14,Dumagami,903,27.8,NaN


In [24]:
smelt.index

Int64Index([1, 2, 4, 5, 6, 8, 10, 11, 13, 14], dtype='int64')

In [25]:
smelt.loc[1,'SourcePort']

916

In [26]:
# Remove mass from inv according the smelt data
# Save all removed mass in a separate smelted DataFrame
smelted = pd.Series(data=0, index=inv.index)
for row in smelt.index:
    for inv_port in inv.columns:
        if str(smelt.loc[row,'SourcePort']) == inv_port:
            if smelt.loc[row,'DMT']>0:
                # Skip the inventory update for the port if it is empty.
                # Because it is impossible to know which material to assign negative mass in the port.
                if (inv.loc[:,inv_port]==0).all(axis=0):
                    print(('Skipped removing inventory from port {} because it is empty.'
                          ' Cannot smelt from an empty port').format(inv_port))
                    continue
                # Calculate remaining fraction in the Eglise Port
                m = 1 - smelt.loc[row,'DMT'] / inv.loc[:,inv_port].sum()
                # Calculate removed mass by shipperID
                smelted += (1-m)*inv.loc[:,inv_port]
                # Update inventory
                inv.loc[:,inv_port] = inv.loc[:,inv_port] * m
                print('Removed {} dmt from port {}'.format(smelt.loc[row,'DMT'], inv_port))
                if m<0:
                    print('Caution, negative inventory in port {}: {} dmt.'.format(inv_port, inv.loc[:,inv_port].sum()))
                
                
inv.sum(axis=0)
        

Removed 704.4 dmt from port 916
Caution, negative inventory in port 916: -195.77999999999994 dmt.
Removed 120.5 dmt from port 900
Removed 253.2 dmt from port 919
Removed 546.2 dmt from port 906
Removed 418.4 dmt from port 918
Caution, negative inventory in port 918: -28.460000000000015 dmt.
Removed 68.4 dmt from port 924
Removed 43.0 dmt from port 915
Removed 49.9 dmt from port 920
Removed 418.4 dmt from port 921
Caution, negative inventory in port 921: -84.01999999999998 dmt.
Removed 27.8 dmt from port 903


564      44.42
560       0.00
900    2645.62
903      83.67
906      26.52
910       0.00
913     436.35
915     241.30
916    -195.78
917     424.00
918     -28.46
919     249.02
920     267.17
921     -84.02
922     150.00
923     147.44
924     109.96
925     318.38
dtype: float64

Save smelted mass by shipperID

In [27]:
#smelted.to_csv('smelted'+date.today().strftime('%d-%m-%Y')+'.csv')

Show smelted materials

In [28]:
smelted

XKM                         30.189037
FLI                         19.527907
MTA                        692.150415
COPA                        16.553873
NAFT                        34.717447
Pyrite_don_rouyn_gallen      0.000000
BCM                          6.708675
DUM                         27.800000
NVA                        107.427317
LSN_et_INC4                  0.000000
TIZ                          0.000000
FAS                         57.623736
EAGL                       597.572952
NMP                        110.084013
STAL                       186.843711
DDG                          0.000000
FDN                         63.414758
CUZ                        313.153748
CLH                          0.000000
BOLA                         0.000000
HGC                         32.060809
CAPE                         0.000000
TMO                         80.161682
UTE                          2.591390
ARUB                        33.564808
IMMSA                       60.087193
CEZ         

In [29]:
# Check if total smelted mass matches the Rapport du reacteur
smelted.sum()

2650.2

In [35]:
# Calculate smelted concentrate mass
# Group smelted categories
smeltCat = pd.read_csv(r'C:\Users\markz\OneDrive\Documents\Fonte_data\smeltCategories.csv')
smeltCat.head()

,Category,shipperCode
0,Conc Vert,XKM
1,Conc Vert,FLI
2,Conc Vert,MTA
3,Pyrite,Pyrite_don_rouyn_gallen
4,Conc Vert,BCM


In [53]:
# Remove null rows
smeltCat= smeltCat[smeltCat['Category'].notnull()]
smeltCat.head()

,Category,shipperCode
0,Conc Vert,XKM
1,Conc Vert,FLI
2,Conc Vert,MTA
3,Pyrite,Pyrite_don_rouyn_gallen
4,Conc Vert,BCM


In [45]:
# Make a dictionnary
smeltCat_dict = smeltCat.set_index('shipperCode')['Category'].to_dict()
smeltCat_dict
#shipID_dict = shipID.set_index('SourcePort')['ShipperID'].to_dict()

{'XKM': 'Conc Vert',
 'FLI': 'Conc Vert',
 'MTA': 'Conc Vert',
 'Pyrite_don_rouyn_gallen': 'Pyrite',
 'BCM': 'Conc Vert',
 'DUM': 'Conc Vert',
 'NVA': 'NVA',
 'LSN_et_INC4': 'Conc Ni',
 'TIZ': 'TIZ',
 'FAS': 'Conc Ni',
 'EAGL': 'Conc Ni',
 'STAL': 'STAL',
 'DDG': 'DDG',
 'FDN': 'FDN',
 'CUZ': 'CUZ',
 'CLH': 'CLH',
 'BOLA': 'BOLA',
 'HGC': 'HGC',
 'Capela': 'Capela',
 'TMO': 'TMO'}

In [46]:
smelted = pd.DataFrame(smelted, columns=['DMT'])

In [52]:
# Map categories to shipperCodes in smelted
#dT['shipperID'] = dT['sourcePort'].map(shipID_dict)
smelted['Category'] = smelted.index.map(smeltCat_dict)
final = smelted.groupby('Category').sum().sort_values('DMT', ascending=False)
final

,DMT
Category,
Conc Vert,776.376035
Conc Ni,655.196687
CUZ,313.153748
STAL,186.843711
NVA,107.427317
TMO,80.161682
FDN,63.414758
HGC,32.060809
BOLA,0.000000


In [51]:
final.sum()

DMT    2214.634748
dtype: float64

In [ ]:
smelt['DMT'].sum()

In [ ]:
'smelted'+date.today().strftime('%d-%m-%Y')+'.csv'